# Задание 3. Выявление лучшей темы рассылки

Расчёт следующих метрик для email-рассылок
* Delivery rate
* Open rate
* Click to Open rate
* Unsubscribe rate
  
Выявление лучшей темы на основе рассчитанных метрик.

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
sheet_name = 'Data' 
spreadsheet_id = '1a1qzLKsks9b31k9H61yg5T5J2aRGbmjAiSg9q396E1w'

url = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df = pd.read_csv(url)
df.info()

selected_columns = ['Название рассылки', 'Дата', 'Тема письма ', 'Отправлено',
                    'Доставлено','Открытия','Клики','Отписки',
                    'Воронка продаж. Шаг 1','Воронка продаж. Шаг 2','Воронка продаж. Шаг 3']
df_selected = df.loc[:,selected_columns]
df_selected.rename(columns={'Название рассылки':'mailing_name',
                           'Дата': 'date', 
                            'Тема письма ': 'topic',
                            'Отправлено':'sent',
                            'Доставлено':'delivered',
                            'Открытия':'openings',
                            'Клики':'clicks',
                            'Отписки':'unsubscribes',
                            'Воронка продаж. Шаг 1':'step_1',
                            'Воронка продаж. Шаг 2':'step_2',
                            'Воронка продаж. Шаг 3':'step_3'},inplace=True)
df_selected.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Название рассылки       218 non-null    object
 1   Название кампании       218 non-null    object
 2   Направление             218 non-null    object
 3   Месяц                   218 non-null    object
 4   Дата                    218 non-null    object
 5   Год                     218 non-null    int64 
 6   Месяц им.               218 non-null    object
 7   Номер недели            218 non-null    int64 
 8   Номер дня               218 non-null    int64 
 9   День недели             218 non-null    object
 10  Время                   218 non-null    object
 11  Веб-версия              218 non-null    object
 12  Тема письма             218 non-null    object
 13  Сегмент                 218 non-null    object
 14  Отправлено              218 non-null    object
 15  Достав

,mailing_name,date,topic,sent,delivered,openings,clicks,unsubscribes,step_1,step_2,step_3
0,Название рассылки 115,15.04.2021,Тема письма 115,688 566,654 138,94 261,3 676,4 514,659,551,435
1,Название рассылки 116,21.04.2021,Тема письма 116,627 527,596 151,83 342,6 001,4 113,2 428,1 971,1 479
2,Название рассылки 117,22.04.2021,Тема письма 117,1 886 230,1 791 919,285 811,34 297,12 364,17 664,17 310,12 637
3,Название рассылки 118,23.04.2021,Тема письма 118,2 342 323,2 225 207,363 154,32 684,15 354,11 634,10 121,7 996
4,Название рассылки 119,30.04.2021,Тема письма 119,724 212,688 001,99 141,8 328,4 747,2 677,2 275,1 888


In [3]:
df_selected.isna().sum()

mailing_name    0
date            0
topic           0
sent            0
delivered       0
openings        0
clicks          0
unsubscribes    0
step_1          0
step_2          0
step_3          0
dtype: int64

In [4]:
df_selected['topic'].nunique()

218

Первичный анализ показал: некорректное наименование столбцов, отсутствуют пропущенные значения, все темы уникальны, некоторые столбцы в некорректном формате.

Приведем данные к корректным форматам.

In [6]:
df_selected['date'] = pd.to_datetime(df_selected['date'], dayfirst=True)
numeric_columns = [ 'sent','delivered', 'openings',
                   'clicks','unsubscribes',
                   'step_1','step_2','step_3'] 

for col in numeric_columns:
    if col in df_selected.columns:
        # Удаляем все нецифровые символы, кроме минуса и точки
        df_selected[col] = df_selected[col].astype(str).str.replace(r'[^\d.-]', '', regex=True)
        df_selected[col] = pd.to_numeric(df_selected[col], errors='coerce')
        
conversion_dict = {
    'sent': 'int',           
    'delivered': 'float',
    'openings': 'float',
    'clicks': 'float',
    'unsubscribes': 'float',
    'step_1': 'float',
    'step_2': 'float',
    'step_3': 'float' 
}

df_selected = df_selected.astype(conversion_dict)
df_selected.info()
df_selected.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   mailing_name  218 non-null    object        
 1   date          218 non-null    datetime64[ns]
 2   topic         218 non-null    object        
 3   sent          218 non-null    int32         
 4   delivered     218 non-null    float64       
 5   openings      218 non-null    float64       
 6   clicks        218 non-null    float64       
 7   unsubscribes  218 non-null    float64       
 8   step_1        218 non-null    float64       
 9   step_2        218 non-null    float64       
 10  step_3        218 non-null    float64       
dtypes: datetime64[ns](1), float64(7), int32(1), object(2)
memory usage: 18.0+ KB


,mailing_name,date,topic,sent,delivered,openings,clicks,unsubscribes,step_1,step_2,step_3
0,Название рассылки 115,2021-04-15,Тема письма 115,688566,654138.0,94261.0,3676.0,4514.0,659.0,551.0,435.0
1,Название рассылки 116,2021-04-21,Тема письма 116,627527,596151.0,83342.0,6001.0,4113.0,2428.0,1971.0,1479.0
2,Название рассылки 117,2021-04-22,Тема письма 117,1886230,1791919.0,285811.0,34297.0,12364.0,17664.0,17310.0,12637.0
3,Название рассылки 118,2021-04-23,Тема письма 118,2342323,2225207.0,363154.0,32684.0,15354.0,11634.0,10121.0,7996.0
4,Название рассылки 119,2021-04-30,Тема письма 119,724212,688001.0,99141.0,8328.0,4747.0,2677.0,2275.0,1888.0


Рассчитаем необходимые метрики.

In [8]:
# 2. Расчет требуемых метрик
df_selected['delivery_rate'] = (df_selected['delivered'] / df_selected['sent'] * 100).round(2)
df_selected['open_rate'] = (df_selected['openings'] / df_selected['delivered'] * 100).round(2)
df_selected['click_to_open_rate'] = (df_selected['clicks'] / df_selected['openings'] * 100).round(2)
df_selected['unsubscribe_rate'] = (df_selected['unsubscribes'] / df_selected['delivered'] * 100).round(2)

df_selected.head()

,mailing_name,date,topic,sent,delivered,openings,clicks,unsubscribes,step_1,step_2,step_3,delivery_rate,open_rate,click_to_open_rate,unsubscribe_rate
0,Название рассылки 115,2021-04-15,Тема письма 115,688566,654138.0,94261.0,3676.0,4514.0,659.0,551.0,435.0,95.0,14.41,3.9,0.69
1,Название рассылки 116,2021-04-21,Тема письма 116,627527,596151.0,83342.0,6001.0,4113.0,2428.0,1971.0,1479.0,95.0,13.98,7.2,0.69
2,Название рассылки 117,2021-04-22,Тема письма 117,1886230,1791919.0,285811.0,34297.0,12364.0,17664.0,17310.0,12637.0,95.0,15.95,12.0,0.69
3,Название рассылки 118,2021-04-23,Тема письма 118,2342323,2225207.0,363154.0,32684.0,15354.0,11634.0,10121.0,7996.0,95.0,16.32,9.0,0.69
4,Название рассылки 119,2021-04-30,Тема письма 119,724212,688001.0,99141.0,8328.0,4747.0,2677.0,2275.0,1888.0,95.0,14.41,8.4,0.69


Для выявления лучшей темы рассчитаем показатель, основанный на рассчитанных метриках

In [10]:
def calculate_composite_score(df):
    # Функция для расчета общей оценки
    df_scoring = df.copy()
    
    # НоПриведем метрики к шкале 0-1
    # Для положительных метрик чем выше, тем лучше
    positive_metrics = ['delivery_rate', 'open_rate', 'click_to_open_rate']
    
    for metric in positive_metrics:
        min_val = df_scoring[metric].min()
        max_val = df_scoring[metric].max()
        if max_val > min_val:
            df_scoring[f'{metric}_norm'] = (df_scoring[metric] - min_val) / (max_val - min_val)
        else:
            df_scoring[f'{metric}_norm'] = 0.5
    
    # Для unsubscribe_rate инвертируем
    min_unsub = df_scoring['unsubscribe_rate'].min()
    max_unsub = df_scoring['unsubscribe_rate'].max()
    if max_unsub > min_unsub:
        df_scoring['unsubscribe_rate_norm'] = 1 - (
            (df_scoring['unsubscribe_rate'] - min_unsub) / (max_unsub - min_unsub)
        )
    else:
        df_scoring['unsubscribe_rate_norm'] = 0.5
    
    # Определим ценность кажжой метрики
    weights = {
        'delivery_rate_norm': 0.15,     # Доставляемость
        'open_rate_norm': 0.35,         # Открываемость
        'click_to_open_rate_norm': 0.40, # Качество контента(считаем самым важным показателем)
        'unsubscribe_rate_norm': 0.10   # Удержание 
    }
    
    # Рассчитываем общую оценку
    df_scoring['composite_score'] = 0
    for metric, weight in weights.items():
        df_scoring['composite_score'] += df_scoring[metric] * weight
    
    # Приводим оценку к 0-100
    min_score = df_scoring['composite_score'].min()
    max_score = df_scoring['composite_score'].max()
    if max_score > min_score:
        df_scoring['composite_score_normalized'] = (
            (df_scoring['composite_score'] - min_score) / (max_score - min_score) * 100
        ).round(2)
    else:
        df_scoring['composite_score_normalized'] = 50.0
    
    return df_scoring

df_with_scores = calculate_composite_score(df_selected)

# Сортируем по итогу
df_sorted = df_with_scores.sort_values('composite_score_normalized', ascending=False)

print(f"\n Лучшая тема по взвешенной оценке: {df_sorted.iloc[0]['topic']}")


 Лучшая тема по взвешенной оценке: Тема письма 207


In [11]:
print(f"\n Метрики лучшей темы:")
best_topic_metrics = df_sorted.iloc[0]
print(f"   • Delivery Rate: {best_topic_metrics['delivery_rate']}%")
print(f"   • Open Rate: {best_topic_metrics['open_rate']}%")
print(f"   • Click-to-Open Rate: {best_topic_metrics['click_to_open_rate']}%")
print(f"   • Unsubscribe Rate: {best_topic_metrics['unsubscribe_rate']}%")


 Метрики лучшей темы:
   • Delivery Rate: 98.5%
   • Open Rate: 17.43%
   • Click-to-Open Rate: 12.0%
   • Unsubscribe Rate: 0.28%
